# 7.1 単語分割（Janome、MeCab+NEologd）

- 本ファイルでは、JanomeもしくはMeCab+NEologdを使用して分かち書きします


※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.1 学習目標

1.	機械学習における自然言語処理の流れを理解する
2.	JanomeおよびMeCab+NEologdを用いた形態素解析を実装できるようになる


# 事前準備

- 書籍の指示に従い、本章で使用するデータを用意します



# 1 . 単語へ分割：Tokenizer

分かち書きをする部分を作成します


## 1.1 Janome

公式サイト

https://mocobeta.github.io/janome/


### Janomeのインストール方法

コンソールにて、

- source activate pytorch_p36
- pip install janome

In [20]:
!pip install janome

In [21]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '機械学習が好きです。'

for token in j_t.tokenize(text):
    print(token)


機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


In [22]:
# 単語分割する関数を定義


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


text = '機械学習が好きです。'
print(tokenizer_janome(text))


['機械', '学習', 'が', '好き', 'です', '。']


## 1.2 MeCab

公式サイト

http://taku910.github.io/mecab/

In [23]:
!apt-get update
!sudo apt-get -y install sudo vim git curl file swig

!sudo apt-get -y install mecab

!sudo apt-get -y install libmecab-dev

!sudo apt-get -y install mecab-ipadic-utf8

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 128 kB in 2s (69.8 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
curl is already the newest version (7.81.0-1ubuntu1.18).
file is already the newest version (1:5.41-3ubuntu0.1).
git is already the newest version (1:2.34.1-1ubuntu1.11).
sudo is already the newest version (1.9.9-1ubuntu2.4).
vim is already the newest version (2:8.2.3995-1ubuntu2.19).
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.
Reading package lists... Done
Building dependenc

### MeCab+NEologdのインストール方法

1. MeCabのインストール

sudo apt install mecab

sudo apt install libmecab-dev

sudo apt install mecab-ipadic-utf8


2. NEologd のインストール

git clone https://github.com/neologd/mecab-ipadic-neologd.git

cd mecab-ipadic-neologd

sudo bin/install-mecab-ipadic-neologd

(途中で止まり、
Do you want to install mecab-ipadic-NEologd? Type yes or no.
と聞かれたら、yesと入力)

3. PythonからMeCabを使用できるようにする 

conda install -c anaconda swig

pip install mecab-python3

cd ..

jupyter notebook --port 9999


In [24]:
!cp /etc/mecabrc /usr/local/etc/

In [25]:
import MeCab

m_t = MeCab.Tagger('-Ochasen')
#m_t = MeCab.Tagger(r'-r /etc/mecabrc -d /usr/share/mecab/dic/mecab-ipadic-neologd')


text = '機械学習が好きです。'

print(m_t.parse(text))


機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [26]:
import MeCab

#m_t = MeCab.Tagger('-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd')
m_t = MeCab.Tagger('-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

text = '機械学習が好きです。'

print(m_t.parse(text))


機械学習	キカイガクシュウ	機械学習	名詞-固有名詞-一般		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [27]:
# 単語分割する関数を定義

m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')


def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret


text = '機械学習が好きです。'
print(tokenizer_mecab(text))


['機械学習', 'が', '好き', 'です', '。']


以上